# Entrenamiento y predicción de modelos

In [ ]:
import pandas as pd
import numpy as np
import openpyxl
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Cargar datos
ventas_h = pd.read_csv('data/Ventas_históricas.xlsx', parse_dates=['fecha'])
inventario = pd.read_csv('data/inventario.csv')
ordenes = pd.read_csv('data/ordenes.csv')

# Preprocesamiento de datos
def preprocess_data(ventas):
    ventas['mes'] = ventas['fecha'].dt.month
    ventas['año'] = ventas['fecha'].dt.year
    ventas = ventas.groupby(['año', 'mes', 'producto']).agg({'cantidad': 'sum'}).reset_index()
    return ventas

ventas = preprocess_data(ventas)

# Dividir datos en entrenamiento y prueba
X = ventas[['año', 'mes', 'producto']]
y = ventas['cantidad']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo
modelo = RandomForestRegressor(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

# Evaluación del modelo
y_pred = modelo.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Error absoluto medio: {mae}')

# Predicción para los próximos meses
def forecast_future(model, start_year, end_year):
    future_months = pd.DataFrame({
        'año': np.repeat(np.arange(start_year, end_year+1), 12),
        'mes': list(range(1, 13)) * (end_year - start_year + 1),
        'producto': [1] * 12 * (end_year - start_year + 1)  # Modificar según productos disponibles
    })
    return model.predict(future_months)

predicciones = forecast_future(modelo, 2024, 2025)
print(predicciones)
